## Compilacion

In [101]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: build: File exists
-- The C compiler identification is AppleClang 11.0.0.11000033
-- The CXX compiler identification is AppleClang 11.0.0.11000033
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp

## Creamos matriz de covarianza

In [102]:
from sklearn import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.feature_extraction.text import CountVectorizer
import os

def get_instances(df, max_df=0.90, min_df=0.1):
  text_train = df[df.type == 'train']["review"]
  label_train = df[df.type == 'train']["label"]

  text_test = df[df.type == 'test']["review"]
  label_test = df[df.type == 'test']["label"]

  vectorizer = CountVectorizer(max_df=max_df, min_df=min_df, max_features=5000)

  vectorizer.fit(text_train)

  X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
  X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

  return X_train, y_train, X_test, y_test

# datasets de train y test
df = pd.read_csv("../data/imdb_small.csv")

X_train, y_train, X_test, y_test = get_instances(df)

M = (X_train - X_train.mean(axis=0))

cov_matrix = M.T @ M / (M.shape[0]-1)
cov_matrix = cov_matrix.copy()
cov_matrix.resize((100, 100))

np.savetxt('../results/predict_cov_matrix_size.txt', cov_matrix.shape)
np.savetxt('../results/predict_cov_matrix.txt', cov_matrix)

w, V = np.linalg.eig(cov_matrix)

# A veces aparecen números complejos acá. Los descartamos
w = np.real(w)
V = np.real(V)
print(w)
np.savetxt('../results/predict_eigen_v.txt', w)
np.savetxt('../results/predict_eigen_W.txt', V)

[ 1.19835824e+01 -4.75421630e-01 -4.75421630e-01  1.64794580e+00
  1.64794580e+00 -1.45677411e+00 -1.45677411e+00 -1.91045159e+00
  1.76060734e+00  1.74054146e-01  1.74054146e-01 -1.71795905e+00
  9.48885977e-01  9.48885977e-01  1.60865293e+00 -1.27528600e+00
 -1.27528600e+00  1.42436878e+00  1.42436878e+00 -1.44394647e+00
 -4.18944690e-02 -4.18944690e-02  8.85966427e-01  8.85966427e-01
  4.51822556e-01  4.51822556e-01 -4.17429844e-01 -4.17429844e-01
 -9.69306438e-01 -9.69306438e-01 -7.89726806e-01 -7.89726806e-01
 -1.15475211e+00  1.11789298e+00  1.07541471e+00  1.07541471e+00
  8.73925684e-01  8.73925684e-01 -7.34502053e-01 -7.34502053e-01
 -4.22883464e-01 -4.22883464e-01 -8.79551558e-01  3.14445943e-01
  3.14445943e-01  5.71453245e-01  5.71453245e-01  6.50779686e-01
  6.50779686e-01 -7.31609342e-01  8.88238203e-02  8.88238203e-02
  5.79621407e-01 -6.20665742e-01 -6.20665742e-01 -3.75645299e-01
 -3.75645299e-01  3.26145860e-01  3.26145860e-01  1.57664002e-03
  1.57664002e-03 -4.18879

## Corremos el test que varia eps y num_its

In [103]:
import os

os.system('./../build/tp2')

32512